En este ejercicio, tendréis que crear una clase con el código que usamos en los ejercicios de pair programming de ETL Transformación I y II.

##### Importamos las librerías necesarias

In [39]:
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"

import requests
import pandas as pd
import numpy as np
from datetime import datetime

##### Creamos la clase

In [40]:
class Extraccion_paises:
    def __init__(self, diccionario_paises):
        self.diccionario_paises = diccionario_paises
    
    def extracciones_paises(self):
        # Creamos un df vacío
        df_clima= pd.DataFrame()
        
        for key, value in self.diccionario_paises.items():
            # Extraemos la latitud y la longitud
            lat = value[0]
            lon = value[1]
            pais = key
            producto = 'meteo'

            url = f'http://www.7timer.info/bin/api.pl?lon=-{lon}&lat={lat}&product={producto}&output=json'
            # Llamamos a la Api
            response = requests.get(url)
        
            df = pd.json_normalize(response.json()['dataseries'])
            df['pais'] = key
            df['latitud'] = lat
            df['longitud'] = lon
            # Concatenamos cada df extraído al df vacío inicial
            df_clima = pd.concat([df_clima, df], axis = 0, ignore_index=True)

            if response.status_code == 200:
                print('La peticion se ha realizado correctamente, se ha devuelto el código de estado:',response.status_code,' y como razón del código de estado: ',response.reason)
            elif response.status_code == 402:
                print('No se ha podido autorizar usario, se ha devuelto el código de estado:', response.status_code,' y como razón del código de estado: ',response.reason)
            elif response.status_code == 404:
                print('Algo ha salido mal, el recurso no se ha encontrado,se ha devuelto el código de estado:', response.status_code,' y como razón del código de estado: ',response.reason)
            else:
                print('Algo inesperado ha ocurrido, se ha devuelto el código de estado:', response.status_code,' y como razón del código de estado: ',response.reason)
            
        return df_clima
    
    def desempaquetar_rh(self, df_clima):
        self.df_clima = df_clima

        df_rh = df_clima['rh_profile'].apply(pd.Series)
        df_limpio_rh = pd.DataFrame()

        for col in range(len(df_rh.columns)):
            # Volvemos a aplicar pd.Series a cada fila, ya que convertirá los keys en columnas y los values en filas 
            # y es una de las maneras posibles para extraer su contenido
            desempaque_dicc = df_rh[col].apply(pd.Series)
            
            layer = desempaque_dicc.iloc[0].index[0]
            num = desempaque_dicc.iloc[0].values[0]
            #Construimos el nombre de la columna
            nom = 'rh_'+ layer + '_' + num

            # Creamos una lista que será el contenido de cada columna creada en este loop   
            lista = []
            for fila in range(len(desempaque_dicc.index)):
                
                value = desempaque_dicc.iloc[fila].values[1]
                lista.append(value)

            # Terminamos el loop rellenando nuestro df vacío con el contenido extraído
            df_limpio_rh[nom] = lista
            
        return df_limpio_rh

    def merge_clima_rh(self, df_clima, df_limpio_rh):
        self.df_clima = df_clima
        self.df_limpio_rh = df_limpio_rh
        
        # Queremos sacar los indices en ambos df
        df_clima.reset_index(inplace=True)
        df_limpio_rh.reset_index(inplace=True)

        # Mergeamos por los indices
        df_clima2 = df_clima.merge(df_limpio_rh, on= ['index'])

        # Eliminamos la columna rh_profile
        df_clima2.drop(['rh_profile'], axis=1, inplace=True)
        
        return df_clima2
        
    def desempaquetar_wind(self, df_clima):
        self.df_clima = df_clima

        df_wind = df_clima['wind_profile'].apply(pd.Series)
        df_limpio_wind = pd.DataFrame()

        for col in range(len(df_wind.columns)):
            # Volvemos a aplicar pd.Series a cada fila, ya que convertirá los keys en columnas y los values en filas 
            desempaq_dicc = df_wind[col].apply(pd.Series)
            
            layer = desempaq_dicc.iloc[0,:].index[0]
            num = desempaq_dicc.iloc[0,:].values[0]
            direction = desempaq_dicc.iloc[0,:].index[1]
            speed = desempaq_dicc.iloc[0,:].index[2]
            
            #Construimos el nombre de las columnas
            nom1 = layer + num + '_' + direction
            nom2 = layer + num + '_' + speed

            # Creamos una lista que será el contenido de cada columna creada en este loop   
            lista1 = []
            lista2 = []
            for fila in range(len(desempaq_dicc.index)):

                value_dir = desempaq_dicc.iloc[fila,:].values[1]
                value_sp= desempaq_dicc.iloc[fila,:].values[2]
                
                lista1.append(value_dir)
                lista2.append(value_sp)

            # Terminamos el loop rellenando nuestro df vacío con el contenido extraído
            df_limpio_wind[nom1] = lista1
            df_limpio_wind[nom2] = lista2
        
        return df_limpio_wind

    def merge_clima_wind(self, df_clima, df_limpio_wind):
        self.df_clima = df_clima
        self.df_limpio_wind = df_limpio_wind

        # Queremos sacar los indices en ambos df
        df_limpio_wind.reset_index(inplace=True)
        # Añadimos este paso para que las funciones sean intercambiables de orden y no nos falte o sobre una columna 'index'
        if 'index' not in df_clima.columns:
            df_clima.reset_index(inplace=True)
        
        # Mergeamos por los indices
        df_clima3 = df_clima.merge(df_limpio_wind, on= ['index'])

        # Eliminamos la columna rh_profile
        df_clima3.drop(['index','wind_profile'], axis=1, inplace=True)
        
        return df_clima3


    def merge_attacks_paises_clima(self, df_clima_total):
        self.df_clima_total = df_clima_total

        df_group = df_clima_total.groupby('pais').mean()

        #Ceamos el df de attacks por los paises elegidos
        df_attacks = pd.read_csv('datos/input/attacks_limpieza_completa.csv', index_col = 0)

        df_attacks_limpio = df_attacks[['year', 'type', 'country', 'age', 'species_', 'fecha_limpia', 'fatal',
       'sex']]

        lista_paises = list(df_clima_total['pais'].unique())
        
        df_attacks_paises = df_attacks_limpio[df_attacks_limpio['country'].isin(lista_paises)]

        df_attacks_clima_paises = df_attacks_paises.merge(df_group, left_on=['country'] , right_on=['pais'])
        
        return df_attacks_clima_paises

    def guardar_pkl_csv(self, df_attacks_clima_paises):
        self.df_attacks_clima_paises = df_attacks_clima_paises

        # Añadimos la fecha de hoy para que las siguienes extracciones no se sobreescriban
        fecha_hoy = datetime.now()
        fecha_hoy = datetime.strftime(fecha_hoy, '%Y-%m-%d')
        df_attacks_clima_paises.to_csv(f'datos/output/attacks_clima_paises_{fecha_hoy}.csv')
        df_attacks_clima_paises.to_pickle(f'datos/output/attacks_clima_paises_{fecha_hoy}.pkl')
        

In [41]:
# Definimos nuestro diccionario y creamos una instancia de nuestra clase
diccionario = {'usa':[39.7837304, -100.445882], 
               'australia':[-24.7761086, 134.755], 
               'south africa':[-28.8166236, 24.991639], 
               'new zealand':[-41.5000831, 172.8344077], 
               'papua new guinea':[-5.6816069, 144.2489081]}

In [42]:
api = Extraccion_paises(diccionario)

In [43]:
# Extraemos nuestro df_clima

df_clima = api.extracciones_paises()
df_clima.head(2)

La peticion se ha realizado correctamente, se ha devuelto el código de estado: 200  y como razón del código de estado:  OK
La peticion se ha realizado correctamente, se ha devuelto el código de estado: 200  y como razón del código de estado:  OK
La peticion se ha realizado correctamente, se ha devuelto el código de estado: 200  y como razón del código de estado:  OK
La peticion se ha realizado correctamente, se ha devuelto el código de estado: 200  y como razón del código de estado:  OK
La peticion se ha realizado correctamente, se ha devuelto el código de estado: 200  y como razón del código de estado:  OK


,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais,latitud,longitud
0,3,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 12}, {'layer': '900m...","[{'layer': '950mb', 'direction': 35, 'speed': ...",24,2,8,1014,rain,1,0,45,2,usa,39.78373,-100.445882
1,6,6,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 13}, {'layer': '900m...","[{'layer': '950mb', 'direction': 45, 'speed': ...",22,-1,11,1014,rain,2,0,80,3,usa,39.78373,-100.445882


In [44]:
# Desempaquetamos rh
df_limpio_rh = api.desempaquetar_rh(df_clima)
df_limpio_rh.head(2)

,rh_layer_950mb,rh_layer_900mb,rh_layer_850mb,rh_layer_800mb,rh_layer_750mb,rh_layer_700mb,rh_layer_650mb,rh_layer_600mb,rh_layer_550mb,rh_layer_500mb,rh_layer_450mb,rh_layer_400mb,rh_layer_350mb,rh_layer_300mb,rh_layer_250mb,rh_layer_200mb
0,12,15,15,14,12,11,6,2,0,-1,-1,0,10,7,14,14
1,13,14,15,14,12,10,6,3,1,0,1,3,6,5,6,14


In [45]:
# Mergeo 1
df_clima2 = api.merge_clima_rh(df_clima, df_limpio_rh)
df_clima2.head(2)

,index,timepoint,cloudcover,highcloud,midcloud,lowcloud,wind_profile,temp2m,lifted_index,rh2m,...,rh_layer_650mb,rh_layer_600mb,rh_layer_550mb,rh_layer_500mb,rh_layer_450mb,rh_layer_400mb,rh_layer_350mb,rh_layer_300mb,rh_layer_250mb,rh_layer_200mb
0,0,3,9,-9999,-9999,-9999,"[{'layer': '950mb', 'direction': 35, 'speed': ...",24,2,8,...,6,2,0,-1,-1,0,10,7,14,14
1,1,6,6,-9999,-9999,-9999,"[{'layer': '950mb', 'direction': 45, 'speed': ...",22,-1,11,...,6,3,1,0,1,3,6,5,6,14


In [46]:
# Desempaquetamos wind
df_limpio_wind= api.desempaquetar_wind(df_clima)
df_limpio_wind.head(2)

,layer950mb_direction,layer950mb_speed,layer900mb_direction,layer900mb_speed,layer850mb_direction,layer850mb_speed,layer800mb_direction,layer800mb_speed,layer750mb_direction,layer750mb_speed,...,layer400mb_direction,layer400mb_speed,layer350mb_direction,layer350mb_speed,layer300mb_direction,layer300mb_speed,layer250mb_direction,layer250mb_speed,layer200mb_direction,layer200mb_speed
0,35,2,160,2,175,2,195,2,295,2,...,270,5,270,5,270,6,275,6,280,6
1,45,3,40,2,160,2,175,2,315,2,...,275,5,275,6,275,6,280,6,280,6


In [47]:
# Mergeo 2
df_clima3 = api.merge_clima_wind(df_clima2, df_limpio_wind)
df_clima3.head(2)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_type,...,layer400mb_direction,layer400mb_speed,layer350mb_direction,layer350mb_speed,layer300mb_direction,layer300mb_speed,layer250mb_direction,layer250mb_speed,layer200mb_direction,layer200mb_speed
0,3,9,-9999,-9999,-9999,24,2,8,1014,rain,...,270,5,270,5,270,6,275,6,280,6
1,6,6,-9999,-9999,-9999,22,-1,11,1014,rain,...,275,5,275,6,275,6,280,6,280,6


In [48]:
# Cargaremos internamente el df attacks desde el csv de la carpeta input y mergeamos con df clima
df_attacks_climas_paises = api.merge_attacks_paises_clima(df_clima3)
df_attacks_climas_paises.head(2)

,year,type,country,age,species_,fecha_limpia,fatal,sex,timepoint,cloudcover,...,layer400mb_direction,layer400mb_speed,layer350mb_direction,layer350mb_speed,layer300mb_direction,layer300mb_speed,layer250mb_direction,layer250mb_speed,layer200mb_direction,layer200mb_speed
0,2018,Boating,usa,57.0,White shark,Jun,N,F,97.5,6.4375,...,209.21875,3.578125,202.109375,3.9375,206.40625,4.296875,215.546875,4.71875,246.09375,5.21875
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,97.5,6.4375,...,209.21875,3.578125,202.109375,3.9375,206.40625,4.296875,215.546875,4.71875,246.09375,5.21875


In [51]:
df_attacks_climas_paises.columns #Comprobamos las columnas que esté todo limpio.

Index(['year', 'type', 'country', 'age', 'species_', 'fecha_limpia', 'fatal',
       'sex', 'timepoint', 'cloudcover', 'highcloud', 'midcloud', 'lowcloud',
       'temp2m', 'lifted_index', 'rh2m', 'msl_pressure', 'prec_amount',
       'snow_depth', 'wind10m.speed', 'latitud', 'longitud', 'rh_layer_950mb',
       'rh_layer_900mb', 'rh_layer_850mb', 'rh_layer_800mb', 'rh_layer_750mb',
       'rh_layer_700mb', 'rh_layer_650mb', 'rh_layer_600mb', 'rh_layer_550mb',
       'rh_layer_500mb', 'rh_layer_450mb', 'rh_layer_400mb', 'rh_layer_350mb',
       'rh_layer_300mb', 'rh_layer_250mb', 'rh_layer_200mb',
       'layer950mb_direction', 'layer950mb_speed', 'layer900mb_direction',
       'layer900mb_speed', 'layer850mb_direction', 'layer850mb_speed',
       'layer800mb_direction', 'layer800mb_speed', 'layer750mb_direction',
       'layer750mb_speed', 'layer700mb_direction', 'layer700mb_speed',
       'layer650mb_direction', 'layer650mb_speed', 'layer600mb_direction',
       'layer600mb_speed', '

In [50]:
# Guardamos nuestros datos extraídos en la carpeta output
api.guardar_pkl_csv(df_attacks_climas_paises)